In [1]:
!pip install tensorflow

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.68.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached ml_dtypes-0.4.1-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using 

In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam


# Load your dataset (adjust the path if necessary
df = pd.read_csv('D:/Infosys Springboard/Internship 5.0 Python Tech Stack/Cyber Bullying Detection Project/labelled_comments_5500.csv')

# Preprocessing: Encoding the labels (0 and 1 for non-cyberbullying and cyberbullying)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Splitting the dataset into train and test sets
X = df['preprocessed_comments'].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization: Convert words to tokens and pad sequences for equal length
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 500
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Build the RNN model with LSTM layer
model = Sequential()

# Embedding layer (turns word indices into dense vectors)
model.add(Embedding(input_dim=5000, output_dim=128))

# LSTM layer (Recurrent layer to learn dependencies in the text)
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32, return_sequences=False)))

# Dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Dense layer with a single output node (for binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# You can save the model for later use
model.save('cyberbullying_detection_model_final.h5')


Epoch 1/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 771s 11s/step - accuracy: 0.5893 - loss: 0.6595 - val_accuracy: 0.7464 - val_loss: 0.5071
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 537s 8s/step - accuracy: 0.8385 - loss: 0.3937 - val_accuracy: 0.7373 - val_loss: 0.5653
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 575s 8s/step - accuracy: 0.9242 - loss: 0.2295 - val_accuracy: 0.7591 - val_loss: 0.6315
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 586s 8s/step - accuracy: 0.9457 - loss: 0.1650 - val_accuracy: 0.7427 - val_loss: 0.7453
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 610s 9s/step - accuracy: 0.9348 - loss: 0.1898 - val_accuracy: 0.7482 - val_loss: 0.7551
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 669s 10s/step - accuracy: 0.9536 - loss: 0.1451 - val_accuracy: 0.7527 - val_loss: 0.6842
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 677s 10s/step - accuracy: 0.9725 - loss: 0.1015 - val_accuracy: 0.7445 - val_loss: 0.8513
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 686s 10s/step - accuracy: 0.9769 - loss: 0.0848 - val_accuracy: 0.7427

Accuracy: 74.09%


In [3]:
import pickle


In [5]:
# Save the tokenizer object to a .pkl file
with open('tokenizer_final.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)
print("Tokenizer saved as 'tokenizer_final.pkl'")


Tokenizer saved as 'tokenizer_final.pkl'
